<a href="https://colab.research.google.com/github/erturkkadir/sifirdan/blob/master/Ilac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install chembl_webresource_client

ChEMBL Database, 2 milyondan fazla bilesenin biolojik aktivite kayitlari 

https://www.ebi.ac.uk/chembl/

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
targets.head(5)

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
df = pd.DataFrame.from_dict(res)
df = df[df.standard_value.notna()]
df = df[df.canonical_smiles.notna()]
df_nr = df.drop_duplicates(['canonical_smiles'])
df_nr
len(df.canonical_smiles.unique())

Select and retrieve bioactivity data for Human Acetylcholinesterase (first entry)
We will assign the fifth entry (which corresponds to the target protein, Human Acetylcholinesterase) to the *selected_target* variable

In [ ]:
df.to_csv('acetylcholinesterase_01_bioactivity_data_raw.csv', index=False)
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df2 = df_nr[selection]
df2